# Part 1 - Exploratory Data Analysis (EDA)

We will start with an Exploratory Data Analysis (EDA) of the Vancouver housing dataset.  
It is always a good idea to start with an EDA before designing and training a machine learning algorithm.  
EDA gives us better insight to the data by using statistical and visualization techniques.  

Upon completing this notebook, we should have:  
* Familiarity with [Pandas] and [NumPy] for data management and analysis
* Familiarity with [Matplotlib] and [seaborn] for visualization
* A decent understanding of the characteristics of our dataset
[Pandas]: https://pandas.pydata.org/
[NumPy]: http://www.numpy.org/
[Matplotlib]: https://matplotlib.org/
[seaborn]: https://seaborn.pydata.org/

In [420]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from geopy import Nominatim
import geojson
import folium
from branca.colormap import LinearColormap, StepColormap

%matplotlib inline

## Let's start by loading the data and have a peek at the contents

In [421]:
import glob
all_csvs = []
# load the csv files from all scraping runs
for filename in glob.glob('./data/sf/**/*.csv'):
    all_csvs.append(pd.read_csv(filename))
# combine all dataframes together and drop any duplicate entries
df = pd.concat(all_csvs, ignore_index=True).drop_duplicates()
# save this combined dataframe as csv for safe keeping
df.to_csv('./data/sf/all.csv')
# display first 5 entries of DataFrame
df.head(5) 

,title,address,city,state,postal_code,price,facts and features,real estate provider,url
0,Condo For Sale,550 Davis St UNIT 44,San Francisco,CA,94111,"$1,995,000","3 bds , 2 ba , 1,520 sqft",Sotheby's International Realty,https://www.zillow.com/homedetails/550-Davis-S...
1,Condo For Sale,240 Lombard St APT 437,San Francisco,CA,94111,"$625,000","1 bd , 1 ba , 566 sqft",SimpleListing.com,https://www.zillow.com/homedetails/240-Lombard...
2,Condo For Sale,550 Davis St UNIT 39,San Francisco,CA,94111,"$1,196,000","1 bd , 1 ba , 914 sqft",NaN,https://www.zillow.com/homedetails/550-Davis-S...
3,Condo For Sale,77 Dow Pl APT 701,San Francisco,CA,94107,"$935,000","1 bd , 1.5 ba , 1,022 sqft",Vanguard Properties,https://www.zillow.com/homedetails/77-Dow-Pl-A...
4,House For Sale,807 Francisco St,San Francisco,CA,94109,"$16,900,000","6 bds , 6.5 ba , 6,180 sqft",Compass,https://www.zillow.com/homedetails/807-Francis...


### Our data is now contained in a variable named `df` which is a pandas DataFrame.

## Display some quick stats about the DataFrame
DataFrame has a few built in functions we can call to get a quick summary of the data:  
* `info()` displays a count of all non-null objects and their datatypes  
* `describe()` calculates basic statistics about all numerical values in the DataFrame

In [422]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 839
Data columns (total 9 columns):
title                   538 non-null object
address                 538 non-null object
city                    538 non-null object
state                   538 non-null object
postal_code             538 non-null object
price                   537 non-null object
facts and features      538 non-null object
real estate provider    417 non-null object
url                     538 non-null object
dtypes: object(9)
memory usage: 42.0+ KB


,title,address,city,state,postal_code,price,facts and features,real estate provider,url
count,538,538,538,538,538,537,538,417,538
unique,11,521,6,1,27,314,478,110,522
top,Condo For Sale,2764 Greenwich St,San Francisco,CA,94109,"$1,995,000","1 bd , 1 ba , -- sqft",Zephyr Real Estate,https://www.zillow.com/homedetails/352-Cumberl...
freq,211,2,494,538,40,13,7,42,2


## Reformat price column

In [414]:
import re
def format_price(price):
    """Remove all non-numerical and period"""
    price = str(price)
    non_decimal = re.compile(r'[^\d.]+')
    price = price.replace('.','')
    price = price.replace('K','000')
    price = price.replace('M','000000')
    price_num = None
    try:
        price_num = float(non_decimal.sub('', price))
    except Exception as e:
        print(e)
    finally:
        return price_num
df['price'] = df.price.apply(format_price)

could not convert string to float: 


## Parse `facts and features` column into multiple columns

In [415]:
# TODO: consider studio as 0 beds?
non_decimal = re.compile(r'[^\d.]+')
def parse_beds(string):
    strings = string.lower().split(', ')
    num_beds = None
    for s in strings:
        if "bd" in s:
            try:
                num_beds = float(non_decimal.sub('', s))
            except Exception as e:
                print(s, e)
            finally:
#                 print(f'num_beds = {num_beds}')
                return num_beds

def parse_bath(string):
    strings = string.lower().split(', ')
    num_bath = None
    for s in strings:
        if "ba" in s:
            try:
                num_bath = float(non_decimal.sub('', s))
            except Exception as e:
                print(s, e)
            finally:
#                 print(f'num_bath = {num_bath}')
                return num_bath
def parse_sqft(string):
    strings = string.lower().split(', ')
    sqft = None
    for s in strings:
        if "ft" in s:
            try:
                sqft = float(non_decimal.sub('', s))
            except Exception as e:
                print(s, e)
            finally:
#                 print(f'sqft = {sqft}')
                return sqft
df['bed'] = df['facts and features'].apply(parse_beds)
df['bath'] = df['facts and features'].apply(parse_bath)
df['sqft'] = df['facts and features'].apply(parse_sqft)

-- bds  could not convert string to float: 
-- bds  could not convert string to float: 
-- bds  could not convert string to float: 
-- bds  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  could not convert string to float: 
-- ba  

## Parse `title` column into `property_type`

First let's see if there is a pattern to the titles:

In [416]:
print(df.title.unique())
print(df.title.value_counts())

['Condo For Sale' 'House For Sale' 'Apartment For Sale' 'New Construction'
 'Foreclosure' 'Lot/Land For Sale' 'Coming Soon' 'Co-op For Sale'
 'Auction' 'For Sale by Owner' 'Townhouse For Sale']
Condo For Sale        211
House For Sale        150
Apartment For Sale    100
New Construction       37
Coming Soon            12
Lot/Land For Sale       9
For Sale by Owner       6
Auction                 6
Co-op For Sale          5
Foreclosure             1
Townhouse For Sale      1
Name: title, dtype: int64


Looks like there is a limited amount of unique values, which is good. We can design our parser to catch most cases.

In [417]:
# map property types
property_types = {'Condo For Sale': 'condo', 
                  'House For Sale': 'house', 
                  'Apartment For Sale': 'apartment', 
                  'New Construction': 'new',
                  'Foreclosure': 'foreclosure', 
                   'Lot/Land For Sale': 'lot', 
                  'Coming Soon': 'coming', 
                  'Co-op For Sale': 'coop',
                  'Auction': 'auction', 
                  'For Sale by Owner': None, 
                  'Townhouse For Sale': 'townhouse'}
def parse_property_type(string):
    try:
        property_type = property_types[string]
    except KeyError as e:
        print(e)
        property_type = None
    finally:
        return property_type
df['property_type'] = df['title'].apply(parse_property_type)

In [418]:
df.property_type.value_counts()

condo          211
house          150
apartment      100
new             37
coming          12
lot              9
auction          6
coop             5
townhouse        1
foreclosure      1
Name: property_type, dtype: int64

## Display some quick stats about the DataFrame
DataFrame has a few built in functions we can call to get a quick summary of the data:  
* `info()` displays a count of all non-null objects and their datatypes  
* `describe()` calculates basic statistics about all numerical values in the DataFrame

In [ ]:
df.info()

In [ ]:
df.describe()

### Wow, very large maximum price albeit not suprising. 

In [ ]:
# describe only the 'price' column
df['price'].describe()

## We have the gist of the dataset size and its contents, it's time to go more in depth and Visualize the data.  
We will use `Seaborn` to visualize the data.

# TODO: Find a place to show violin plots
# TODO: consider PCA for visualizing high dimensional data

### Plot histogram of prices

In [ ]:
# globally set our seaborn plot size to 12 by 8 inches:
sns.set(rc={'figure.figsize':(12, 8)})

def plot_prices(df: pd.DataFrame, bins: list):
    fig, ax = plt.subplots()
    ax.set_xticks(bins)
    plt.xticks(rotation='vertical')
    return sns.distplot(df.price, bins=bins)

bins = range(int(df.price.min()), int(df.price.max()), 500000)
plot_prices(df.dropna(), bins)

### Definitely a skewed distribution, looks as if we have a few outliers at the higher range of the prices.  
### We can quantify this by calculating:  
* `Skewness` - A measure of the symmetry (or lack thereof) of a distribution
* `Kurtosis` - Whether distrubition is "heavy-tailed" or "light-tailed" or in other words: how "sharp" the peak is.

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['price'].skew())
print("Kurtosis: %f" % df['price'].kurt())

## Plot with outliers removed

In [ ]:
df_no_outliers = df[df.price < 15e6]
bins = range(int(df_no_outliers.price.min()),int(df_no_outliers.price.max()),500000)
plot_prices(df_no_outliers, bins)
print("Skewness (outliers removed): %f" % df_no_outliers['price'].skew())
print("Kurtosis (outliers removed): %f" % df_no_outliers['price'].kurt())

### Removing the outliers improved our skewness and kurtosis values.
We will remember this when cleaning the data for our model. Machine learning models work best with normally distributed data. Outliers may affect model performance.

## Plot missing values.
Recall that there were some columns which are incomplete. Plot a bar graph describing this:

In [ ]:
missing = df.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

Variables that are missing values can either be removed from the dataset or have their missing values replaced (perhaps with 0 or the mean of the column). Remember this for data cleaning.

## Since we have geolocations of the houses in the `latlng` column, let's visualize the data on a slippy map and see if there are any patterns w.r.t. price.  
We use `folium` to render html in the notebook.  
Note that there are hundreds of houses to be displayed and this requires a lot of RAM. If your browser crashes you can adjust the amount to be displayed by changing the variable `display_max`.

In [ ]:
from tqdm import tqdm_notebook
from geopy.geocoders import Nominatim
def generate_latlng(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    geocoder = Nominatim()
    latlngs = []
    for address, city in zip(tqdm_notebook(dataframe.address), dataframe.city):
        location = geocoder.geocode(f'{address} {city}')
        if location:
            latlngs.append((location.latitude, location.longitude))
        else:
            latlngs.append(None)
    dataframe['latlng'] = latlngs
    return dataframe

In [ ]:
from multiprocessing import Pool
job = Pool().apply_async(func=generate_latlng, args=(df,))

In [ ]:
df = job.get()
df

In [ ]:
def draw_houses_on_map(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    # create a folium map object centered in SF
    m = folium.Map(location=(37.7, -122.4))
    # create a colormap of the prices (we limit prices between 5e5 and 10e6)
    colors = ['gray', 'green','blue','red','orange', 'yellow']
    min_price, max_price = 5e5, 6e6
    colormap = StepColormap(colors=colors,vmin=min_price, vmax=max_price, caption='price')
    m.add_child(colormap)
    # amount of points to render on the map. WARNING: significant RAM required to plot all points and may crash your browser 
    display_max = len(dataframe) # plot all
    # display_max = 100 # uncomment and adjust this number if needed
    displayed = 0
    for i, latlng in zip(dataframe.index, dataframe['latlng']):
        price = dataframe.loc[i, 'price']
        if latlng is not None and latlng != 'MISSING':
            if isinstance(latlng, str):
                lat, lng = latlng.replace('(','').replace(')','').split(',')
                latlng = (float(lat), float(lng))
            if not isinstance(latlng, tuple):
                continue
            style = {'fillColor': colormap(price),
                    'color' : colormap(price)}
            p = geojson.Point(coordinates=(latlng[1], latlng[0]), style=style)
            # build an HTML string to be displayed if we click a marker.
            html_info = '<li>Price: ${}</li><li>Property Type: {}</li>'.format(dataframe.loc[i, 'price'], dataframe.loc[i, 'property_type'])
            m.add_child(folium.Marker(location=latlng, icon=folium.Icon(color='black', icon_color=colormap(price)), popup=folium.Popup(html=html_info)))
            displayed += 1
            if displayed > display_max:
                break
    return m
draw_houses_on_map(df)

### We can observe some patterns w.r.t. location.  
### Seems the more expensive homes are Central and North and the "lower" (finger quotes) priced homes on the outside

## Next, let's see how some of the variables interact with the list price.  
Since `price` is our target variable (the variable we are trying to predict), it is useful visualize how each variable relates to `price`. 

### sqft
Total square footage

In [ ]:
# sqft/saleprice
var = 'sqft'
sns.regplot(df[var], df['price'], )

The relationship looks linear with some spreading as sqft increases. We can also see there are some houses with almost zero square feet! Let's investigate why:  
  
Note on `pandas.DataFrame` indexing:  
* `df['sqft'] == 0` gives us a "truth array" where True values match the condition and False otherwise. If we index the original DataFrame with this truth array we get a filtered result

In [ ]:
# filter the DataFrame with nearly zero sqft
df[df['sqft'] < 10].head()

Looks like we have some bad data from the web scraping. We will remember to remove these when we get to our data cleaning notebook

### bed
Number of bedrooms

In [ ]:
var = 'bed'
sns.regplot(df[var], df['price'], )

We observe a bit of linearity

### bath
Number of bathrooms

In [ ]:
var = 'bath'
sns.regplot(df[var], df['price'], )

Relationship appears to be linear.

## Generate a correlation matrix

A correlation matrix will graphically show us which variables are most correlated to our target variable `price`

## TODO: a good explanation of heatmaps here: https://www.kaggleusercontent.com/kf/2154073/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..KfxeCxv48WLAORmGT131Aw.QwZwhYcY6QENL0zU-rCwVvcyXITzFfs1X-Mum5zodrGahfZ9NSDfaSzF7kbocoMmO0fi8m72nware9xs4bOiUrxe9LH6X4Q0qzoYo8TBY0e0He52L0ORo7BOewfCoVLW.D8GjFZ-DLUlizQZOXlbw-g/__results__.html#Interpreting-The-Heatmap

In [ ]:
corrmat = df.corr()
sns.heatmap(corrmat, vmax=1, square=True);

# TODO: Also explain ordinal variables like here: https://www.kaggleusercontent.com/kf/2154073/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..KfxeCxv48WLAORmGT131Aw.QwZwhYcY6QENL0zU-rCwVvcyXITzFfs1X-Mum5zodrGahfZ9NSDfaSzF7kbocoMmO0fi8m72nware9xs4bOiUrxe9LH6X4Q0qzoYo8TBY0e0He52L0ORo7BOewfCoVLW.D8GjFZ-DLUlizQZOXlbw-g/__results__.html#Types-Of-Features

## Categorical Variables.  
So far we have only dealt with numeric variables however there are several non-numerical (**Categorical**) variables to be investigated as well.  
Categorical variables are ones which provide information but are not quantified numerically. For instance, the `sub_area` variable gives us information about what neighbourhood the house is located in (ie. "Kerrisdale", "Yaletown" etc.). From our map plot, we found this information is important when considering house prices.  
In order to use these categorical variables in our model, we encode them into a numerical representation called a [Dummy Variable]. We cover Dummy Variables in a later notebook.
[Dummy Variable]: https://en.wikipedia.org/wiki/Dummy_variable_(statistics)

In [ ]:
print(df.columns)

Let's choose `property_type`, and `postal_code` to investigate.  
We can use the `unique()` function on the categorical columns to see the different categories.

In [ ]:
# print(df['area'].unique())
print(df['postal_code'].value_counts())
print(df['property_type'].unique())
# print(df['strata_type'].unique())


There are some missing values, let's get rid of that data

In [ ]:
postal_codes = ['94501',
'94608',
'94607',
'94005',
'94014',]
for postal_code in postal_codes:
    df = df[df.postal_code != int(postal_code)]
property_types = ['townhouse', 'foreclosure']
for property_type in property_types:
    df = df[df.property_type != property_type]
print(df['postal_code'].value_counts())
print(df['property_type'].value_counts())

Visualize these categories as box plots.  
We use the `pandas.melt()` function to flatten our variables into a single column so we can plot.  
The result of using `melt()` is most easily understood by displaying the result.

In [ ]:
vars_to_analyze = ['postal_code', 'property_type']
df_melt = pd.melt(df, id_vars=['price'], value_vars=vars_to_analyze)
for var in vars_to_analyze:
    df_var = df_melt[df_melt['variable'] == var]
    sns.boxplot(x=df_var['value'], y=df_var['price'])
    x=plt.xticks(rotation=45)
    plt.title(var)
    plt.show()

## Analysis of variance (ANOVA)
We use ANOVA to explore how much variance occurs **between** groups (ie. *[price vs sub_area]* vs *[price vs area]* vs *[price vs property_type]* vs *[price vs strata_type]*) versus how much variance occurs **within** each group (ie *price vs sub_area* alone).  
In the end this tells us is how useful it will be to group `price` into these 4 groups (and if including each variable in our model is useful to us).  
Here's a quick YouTube video that may better explain ANOVA:  

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(id='ITf4vHhyGpc')

In [ ]:
def anova(df):
    anv = pd.DataFrame()
    anv['feature'] = vars_to_analyze
    pvals = []
#     import pdb; pdb.set_trace()
    for c in vars_to_analyze:
        samples = []
        for cls in df[c].unique():
            s = df[df[c] == cls]['price'].values
            samples.append(s)
        try:
            print(samples)
            pval = stats.f_oneway(*samples)[1]
            print(pval)
        except Exception as e:
            pval=None
            print(e)
        finally:
            pvals.append(pval)
    anv['pval'] = pvals
    return anv.sort_values('pval')

a = anova(df)
a['disparity'] = np.log(1./a['pval'].values)
sns.barplot(data=a, x='feature', y='disparity')
x=plt.xticks(rotation=45)

This gives us a rough estimate of effect each variable will have on our model. It makes intuitive sense that `sub_area` is highest since a home in Point Grey is likely to be more expensive than one in Grandview. 

## Hopefully the EDA has improved our intuition about the dataset. Now we can move onto data cleaning!